### Configuration

In [1]:
import os
import re
import numpy as np
import pandas as pd

import nibabel as nib
import numpy.matlib as matlib
from scipy.interpolate import interpn

In [2]:
import utils__config
os.chdir(utils__config.working_directory)
os.getcwd()

'g:\\My Drive\\Residency\\Research\\Lab - Damisah\\Project - Sleep'

### Parameters

**We are using a 2mm brain for voxel placement; this will reduce the accuracy. So try to source a 1mm MNI152 T1 brain**

In [7]:
elec_path = 'Cache/Subject01/S01_electrodes_tkrRAS.csv'
template_path = 'Data/Brain_Templates/MNI152_T1_2mm_brain.nii.gz'
#template_path = 'Data/Brain_Templates/spm152.nii.gz'
output_path = 'Cache/Subject01/S01_electrodes.nii.gz'

### Convert tkrRAS -> Voxels and save as NIfTI

Adapted from: https://github.com/jadecci/coord2nii

In [8]:
# Load electrodes, convert to numpy, and transpose
elec = pd.read_csv(elec_path)
coords = elec[['Coord_x', 'Coord_y', 'Coord_z']].to_numpy().T

# Convert MNI152? to RAS (only if using MNI152? coords)
# coords[0, :] = - coords[0, :]

# Load MRI template (must be in MNI space)
template = nib.load(template_path)
vox2ras = template.get_sform()
data = template.get_data()

# Convert tkrRAS -> Voxel coordinates
ras_centered = coords - matlib.repmat(vox2ras[0:3, 3], coords.shape[1], 1).T
vox_coords = np.linalg.inv(vox2ras[0:3, 0:3]) @ ras_centered

# Find the nearest voxel for the voxel coordinates
grid_pts = [range(data.shape[0]), range(data.shape[1]), range(data.shape[2])]
linear_ind = np.arange(np.prod(data.shape)).reshape(data.shape)
vox_ind = interpn(grid_pts, linear_ind, vox_coords.T, 'nearest')

# Write a value of 1 to each selected voxel
vol = np.zeros(data.shape)
vol[np.unravel_index(vox_ind.astype(int), data.shape)] = 1

# Save the new volume as a NIfTI file
img = nib.Nifti1Image(vol, template.affine, template.header)
nib.save(img, output_path)

C:\Users\Layton\AppData\Local\Temp\ipykernel_324\2071448565.py:11: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = template.get_data()


ValueError: One of the requested xi is out of bounds in dimension 0